In [1]:
-- load libraries
require 'torch'
require 'nn'
require 'nnx'
require 'optim'
require 'image'
-- require 'dataset-mnist'
require 'pl'
require 'paths'
require 'cunn'
npy4th = require 'npy4th'
------------------------------------------------------------------------------
opt = {
        save = 'logs',          -- file to save network after each epoch
        batchSize = 10,         -- mini-batch size
        learningRate = 0.05,    -- learning rate, for SGD only
        momentum = 0,           -- momentum, for SGD only
        max_epochs = 30,        -- Number of epochs to train the model for
        optimization = 'SGD',   -- optimization: SGD | LBFGS
        maxIter = 3,            -- maximum nb of iterations per batch, for LBFGS
        coefL1 = 0,             -- L1 penalty on the weights
        coefL2 = 0,             -- L2 penalty on the weights
        plot = true,            -- plot while training
}

-- fix seed
-- torch.manualSeed(42)

/home/sam/torch/install/share/lua/5.1/cunn/init.lua:1: module 'cutorch' not found:
	no field package.preload['cutorch']
	no file '/home/sam/.luarocks/share/lua/5.1/cutorch.lua'
	no file '/home/sam/.luarocks/share/lua/5.1/cutorch/init.lua'
	no file '/home/sam/torch/install/share/lua/5.1/cutorch.lua'
	no file '/home/sam/torch/install/share/lua/5.1/cutorch/init.lua'
	no file './cutorch.lua'
	no file '/home/sam/torch/install/share/luajit-2.1.0-beta1/cutorch.lua'
	no file '/usr/local/share/lua/5.1/cutorch.lua'
	no file '/usr/local/share/lua/5.1/cutorch/init.lua'
	no file '/home/sam/torch/install/lib/cutorch.so'
	no file '/home/sam/.luarocks/lib/lua/5.1/cutorch.so'
	no file '/home/sam/torch/install/lib/lua/5.1/cutorch.so'
	no file './cutorch.so'
	no file '/usr/local/lib/lua/5.1/cutorch.so'
	no file '/usr/local/lib/lua/5.1/loadall.so'
stack traceback:
	[C]: in function 'require'
	/home/sam/torch/install/share/lua/5.1/cunn/init.lua:1: in main chunk
	[C]: in function 'require'
	[string "-- load

### define train model

In [2]:
function cnn()
    -- use floats for SGD
    torch.setdefaulttensortype('torch.FloatTensor')
    -- define model to train
    classes = {'1','2'}
    -- geometry: width and height of input images
    geometry = {17,50}
    model = nn.Sequential()
    -- convolutional network 
    -- stage 1 : mean suppresion -> filter bank -> squashing -> max pooling
    -- MM constructs the Toeplitz matrix and does matrix multiplication
    -- Now the best option is to use SpatialConvolutionMM on both CPU and GPU!
    -- It's the fastest, it doesn't have any restrictions wrt sizes (SpatialConvolutionCUDA only supports square sizes)
    model:add(nn.SpatialConvolutionMM(1, 32, 5, 5))
    -- model:add(nn.SpatialConvolutionMM(1, 64, 5, 5, 1, 1, 2, 2))
    model:add(nn.Tanh())
    model:add(nn.SpatialMaxPooling(3, 3, 3, 3, 1, 1))
    -- model:add(nn.SpatialMaxPooling(3, 3, 3, 3, 1, 1))
    -- stage 2 : mean suppresion -> filter bank -> squashing -> max pooling
    model:add(nn.SpatialConvolutionMM(32, 64, 5, 5))
    -- model:add(nn.SpatialConvolutionMM(64, 64, 3, 3, 1, 1, 1, 1))
    model:add(nn.Tanh())
    -- model:add(nn.ReLU())
    model:add(nn.SpatialMaxPooling(2, 2, 2, 2))
    -- stage 3 : standard 2-layer MLP:
    model:add(nn.Reshape(64*3*3))
    -- model:add(nn.Reshape(64*5*5))
    model:add(nn.Linear(64*3*3, 200))
    -- model:add(nn.Linear(64*5*5, 200))
    -- model:add(nn.Dropout(0.5)) -- A
    model:add(nn.Tanh())
    -- model:add(nn.ReLU())
    model:add(nn.Linear(200, #classes))
    -- softmax layer
    model:add(nn.LogSoftMax())
--      model:add(nn.SoftMax())

    -- use model on gpu
    model = model:cuda()

    -- retrieve parameters and gradients
    parameters,gradParameters = model:getParameters()

    -- -- loss function
    criterion = nn.ClassNLLCriterion()
    criterion = criterion:cuda()
end

### create train and test dataset and normalize

In [3]:
function prepare_data()
    -- nbTrainingPatches = 60000
    -- nbTestingPatches = 10000
    nbTrainingPatches = 2000
    nbTestingPatches = 1000

    -- loading index
    bag_n = torch.randperm(nbTrainingPatches)[{ {1,1800} }]:long()
    -- create training set and normalize
    trainData = mnist.loadTrainSet(nbTrainingPatches, bag_n, geometry)
    trainData:normalizeGlobal(mean, std)
    -- create test set and normalize
    testData = mnist.loadTestSet(nbTestingPatches, _, geometry)
    testData:normalizeGlobal(mean, std)

--     -- take a quick look of the data
--     itorch.image(trainData.data[100]) -- display the 100-th image in train
--     print(classes[trainData.labels[100]]-1)
--     itorch.image(testData.data[100]) -- display the 100-th image in test
--     print(classes[testData.labels[100]]-1)
end

### Prepare data to use GPU

In [4]:
function convert_gpu()
    trainData.data = trainData.data:cuda()
    trainData.labels = trainData.labels:cuda()

    testData.data = testData.data:cuda()
    testData.labels = testData.labels:cuda()
end

In [5]:
-- print(trainData)
-- print(testData)

### define training and testing functions

In [6]:
function conf_log(n)
    -- this matrix records the current confusion across classes
    confusion = optim.ConfusionMatrix(classes)
    -- log results to files
    name_acc = 'accuracy' .. tostring(n) .. '.log'
    name_err = 'err' .. tostring(n) .. '.log'
    accLogger = optim.Logger(paths.concat(opt.save, name_acc))
    errLogger = optim.Logger(paths.concat(opt.save, name_err))
end

### training function

In [7]:
-- training function
function train(dataset)
    -- epoch tracker
    epoch = epoch or 1

    -- local vars
    local time = sys.clock()
    local trainError = 0
    
    -- do one epoch
--     print('<trainer> on training set:')
--     print('<trainer> online epoch # ' .. epoch .. ' [batchSize = ' .. opt.batchSize .. ']')
    for t = 1,dataset:size(),opt.batchSize do
        -- create mini batch
        local inputs = torch.Tensor(opt.batchSize,1,geometry[1],geometry[2])
        local targets = torch.Tensor(opt.batchSize)
        -- convert to cuda tensor
        inputs = inputs:cuda()
        targets = targets:cuda()
        local k = 1
        for i = t,math.min(t+opt.batchSize-1,dataset:size()) do
            -- load new sample
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        -- create closure to evaluate f(X) and df/dX
        local feval = function(x)
            -- just in case:
            collectgarbage()

            -- get new parameters
            if x ~= parameters then
                parameters:copy(x)
            end

            -- reset gradients
            gradParameters:zero()

            -- evaluate function for complete mini batch
            local outputs = model:forward(inputs)
            local f = criterion:forward(outputs, targets)

            -- train error
            trainError = trainError + f
            
            -- estimate df/dW
            local df_do = criterion:backward(outputs, targets)
            model:backward(inputs, df_do)

            -- penalties (L1 and L2):
            if opt.coefL1 ~= 0 or opt.coefL2 ~= 0 then
                -- locals:
                local norm,sign= torch.norm,torch.sign

                -- Loss:
                f = f + opt.coefL1 * norm(parameters,1)
                f = f + opt.coefL2 * norm(parameters,2)^2/2

                -- Gradients:
                gradParameters:add( sign(parameters):mul(opt.coefL1) + parameters:clone():mul(opt.coefL2) )
            end

            -- update confusion
            for i = 1,opt.batchSize do
                confusion:add(outputs[i], targets[i])
            end

            -- return f and df/dX
            return f,gradParameters
        end

        -- optimize on current mini-batch
        if opt.optimization == 'LBFGS' then

            -- Perform LBFGS step:
            lbfgsState = lbfgsState or {
                maxIter = opt.maxIter,
                lineSearch = optim.lswolfe
            }
            optim.lbfgs(feval, parameters, lbfgsState)

            -- disp report:
            print('LBFGS step')
            print(' - progress in batch: ' .. t .. '/' .. dataset:size())
            print(' - nb of iterations: ' .. lbfgsState.nIter)
            print(' - nb of function evalutions: ' .. lbfgsState.funcEval)

        elseif opt.optimization == 'SGD' then

        -- Perform SGD step:
        sgdState = sgdState or {
            learningRate = opt.learningRate,
            momentum = opt.momentum,
            learningRateDecay = 5e-7
        }
        optim.sgd(feval, parameters, sgdState)

        -- disp progress
        xlua.progress(t, dataset:size())

        else
            error('unknown optimization method')
        end
    end
   
    -- time taken
    time = sys.clock() - time
    time = time / dataset:size()
--     print("<trainer> time to learn 1 sample = " .. (time*1000) .. 'ms')

    -- print confusion matrix
--     print(confusion)
    
    -- train error
    trainError = trainError / math.floor(dataset:size()/opt.batchSize)
    -- train accuracy
    local trainAccuracy = confusion.totalValid * 100
    confusion:zero()

    -- next epoch
    epoch = epoch + 1
    return trainAccuracy, trainError
end

### Test function

In [8]:
-- test function
function test(dataset)
    -- local vars
    local time = sys.clock()
    local testError = 0
    -- define pred for all test data
    local pred_total = torch.zeros(opt.batchSize):long()
    
    -- test over given dataset
--     print('<trainer> on testing Set:')
    for t = 1,dataset:size(),opt.batchSize do
        -- disp progress
        xlua.progress(t, dataset:size())

        -- create mini batch
        local inputs = torch.Tensor(opt.batchSize,1,geometry[1],geometry[2])
        local targets = torch.Tensor(opt.batchSize)

        -- convert to cuda tensor
        inputs = inputs:cuda()
        targets = targets:cuda()
        
        local k = 1
        for i = t,math.min(t+opt.batchSize-1,dataset:size()) do
            -- load new sample
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end

        -- test samples
        local preds = model:forward(inputs)
        local maxs, indices = torch.max(preds, 2)
        indices = indices:long()
        pred_total = torch.cat(pred_total, indices[{{}, 1}])
        
        -- compute error
        err = criterion:forward(preds, targets)
        testError = testError + err   

        -- confusion:
        for i = 1,opt.batchSize do
            confusion:add(preds[i], targets[i])
        end
    end

    -- timing
    time = sys.clock() - time
    time = time / dataset:size()
--     print("<trainer> time to test 1 sample = " .. (time*1000) .. 'ms')

    -- print confusion matrix
--     print(confusion)
    -- test error
    testError = testError / math.floor(dataset:size()/opt.batchSize)
    -- test accuracy
    local testAccuracy = confusion.totalValid * 100
    confusion:zero()

    return testAccuracy, testError, pred_total[{{opt.batchSize+1, pred_total:size()[1]}}]
end

### train out model!

In [9]:
-- -- bagging
-- n_seed = 42
-- N_bags = 20
-- pred_bag = torch.zeros(1, 1000):long() -- initialize pred
-- for i = 1, N_bags do
--     torch.manualSeed(n_seed)
--     print('Ensembling >>>>>>>>' .. i)
--     -- init 
--     cnn()
--     prepare_data()
--     convert_gpu()
--     conf_log(n_seed)
    
--     -- train
--     n = 0
--     while n < opt.max_epochs do
--         n = n + 1
--         trainAcc, trainErr = train(trainData)
--         testAcc, testErr, pred_n = test(testData)
        
--         if opt.plot then
--             -- update logger
--             accLogger:add{['% (mean)train accuracy'] = trainAcc, ['% (mean)test accuracy'] = testAcc}
--             errLogger:add{['% (mean)train error'] = trainErr, ['% (mean)test error'] = testErr}
--             -- plot logger
-- --             accLogger:style{['% (mean)train accuracy'] = '+-', ['% (mean)test accuracy'] = '+-'}
-- --             errLogger:style{['% (mean)train error']    = '+-', ['% (mean)test error']    = '+-'}
-- --             accLogger:plot()
-- --             errLogger:plot()
--         end
--     end 
--     pred_n = pred_n:resize(1, pred_n:size()[1])
--     pred_bag = torch.cat(pred_bag, pred_n, 1)
--     n_seed = n_seed + i
--     print("Done " .. i)
-- end

-- pred_bag = pred_bag[{ {2,N_bags}, {} }]
-- -- use majority vote
-- ensembled_pred, _ =  torch.mode(pred_bag, 1)

-- -- apply confusion matrix
-- confusion_f = optim.ConfusionMatrix(classes)
-- for i = 1, 1000 do
--     forward_pred = -1*torch.ones(10)
--     ind = ensembled_pred[{1, {i}}][1]
--     forward_pred[{{ind}}] = -0.5
--     confusion_f:add(forward_pred, testData.labels[{i}])
-- end

-- print(confusion_f)
-- -- local final_accuracy = confusion_f.totalValid * 100
-- final_accuracy = confusion_f.totalValid * 100
-- print('The final ensembled (mean) accuracy is: ' .. final_accuracy)